In [1]:
# imports
from dotenv import load_dotenv
import openai
import os

from langchain.document_loaders import WebBaseLoader, UnstructuredFileLoader, TextLoader, DirectoryLoader, PyPDFLoader, UnstructuredXMLLoader

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_google_vertexai import VertexAI

from langchain import PromptTemplate, LLMChain
from langchain.document_loaders.merge import MergedDataLoader

from IPython.display import Markdown as md

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv(".env")

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

from langchain_google_vertexai.model_garden import ChatAnthropicVertex

model_name="gemini-2.5-pro-exp-03-25"
llm = VertexAI(
    model_name=model_name,
    temperature=0,
    frequency_penalty=0,
    presence_penalty=0,
    top_p=1.0,)

USER_AGENT environment variable not set, consider setting it to identify your requests.
Unexpected argument 'frequency_penalty' provided to VertexAI.
Unexpected argument 'presence_penalty' provided to VertexAI.


In [2]:
def load_and_construct(directory_paths: list, file_extensions: list) -> str:
    """
    Lädt Dateien aus einer Liste von Verzeichnissen mit bestimmten Dateiendungen,
    extrahiert Metadaten und Inhalt und gibt einen zusammengefassten String zurück.

    Args:
        directory_paths (list): Eine Liste von Pfaden zu den Verzeichnissen.
        file_extensions (list): Eine Liste der zu ladenden Dateiendungen (ohne Punkt).

    Returns:
        str: Ein String, der die Metadaten und Inhalte der Dateien zusammenfasst.
    """

    all_loaders = []
    for directory_path in directory_paths:
        for ext in file_extensions:
            all_loaders.append(
                DirectoryLoader(
                    path=directory_path,
                    glob=f"**/*.{ext}",  # Verwende "**/" für rekursive Suche in allen Unterverzeichnissen
                    recursive=True,
                    loader_cls=TextLoader,
                    show_progress=True # Optional: Zeigt einen Fortschrittsbalken an
                )
            )

    merged_loader = MergedDataLoader(loaders=all_loaders)
    raw_documents = merged_loader.load()

    result_string = ""
    for document in raw_documents:
        # Extrahiere den Dateinamen aus dem vollständigen Pfad in den Metadaten
        filepath = document.metadata["source"]
        result_string += f"Filepath: {filepath}\n"
        result_string += f"{document.page_content}\n"
        result_string += "=====================================\n"

    return result_string

In [13]:
# src_dir = load_and_construct(["/home/randy/Projekte/gdg/vertex-ai/"], ["py", "md"])

readme_loader = TextLoader("/home/randy/Projekte/gdg/vertex-ai/README.md")
readme = readme_loader.load()[0].page_content

prompt_template = """
Ich habe hier eine README.md mit Snippets. Schreibe einen Erläuternden Text, der die Funktionsweise des Codes erklärt. Mehr soll nicht in die Datei. Ich möchte nur ein paar erläuternde Worte zu den Snippets.

Dabei sollen die eigentlich Snippets in dem Text enthalten sein. Vor den jeweiligen Zeilen sollte stehen, was jetzt gemacht wird.

{readme}

Deine Antwort auf englisch im Markdown-Format:
"""

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template),
    verbose=True
)

output = llm_chain.predict(readme=readme)

from datetime import datetime

# Erhalte den aktuellen Zeitstempel
timestamp = datetime.now().strftime('%Y-%m-%d-%H%M')


# Erstelle den Dateinamen mit dem Zeitstempel
filename = f'output/{timestamp}_tutorial_{model_name}.md'


# Öffne die Datei und schreibe den Inhalt hinein
with open(filename, 'w') as file:
    file.write(output)
    



> Entering new LLMChain chain...
Prompt after formatting:

Ich habe hier eine README.md mit Snippets. Schreibe einen Erläuternden Text, der die Funktionsweise des Codes erklärt. Mehr soll nicht in die Datei. Ich möchte nur ein paar erläuternde Worte zu den Snippets.

Dabei sollen die eigentlich Snippets in dem Text enthalten sein. Vor den jeweiligen Zeilen sollte stehen, was jetzt gemacht wird.

# Snippets

```bash
python3.11 -m venv venv
source venv/bin/activate
pip install -r requirements.txt
```

```
gcloud auth activate-service-account --key-file=/home/randy/Projekte/privat/randy-gupta-workspace/terraform/randy-gupta-workspace-beae37726ae2.json
gcloud config set project randy-gupta-workspace
gcloud config set billing/quota_project randy-gupta-workspace
```

```
export GOOGLE_CLOUD_QUOTA_PROJECT="randy-gupta-workspace"
export GOOGLE_APPLICATION_CREDENTIALS="/home/randy/Projekte/privat/randy-gupta-workspace/terraform/randy-gupta-workspace-beae37726ae2.json"
```



Deine Antwort auf